In [2]:
from spotlight.cross_validation import random_train_test_split
from spotlight.datasets.movielens import get_movielens_dataset
from spotlight.evaluation import mrr_score,precision_recall_score
from spotlight.factorization.implicit import ImplicitFactorizationModel
import pandas as pd
import numpy as np
import csv
from spotlight.interactions import Interactions
from spotlight.torch_utils import cpu, gpu, minibatch, set_seed, shuffle
import torch
import torch.nn as nn
import math
from torch.autograd import Variable, Function
import torch.nn as nn

import random


In [4]:
books_ratings_df = pd.read_csv('data/BX-CSV-Dump/BX-Book-Ratings.csv',delimiter=';', encoding = "ISO-8859-1")
books_df = pd.read_csv('data/BX-CSV-Dump/BX-Books.csv',delimiter=';',encoding = "ISO-8859-1",quoting=csv.QUOTE_ALL,error_bad_lines=False )
users_df = pd.read_csv('data/BX-CSV-Dump/BX-Users.csv',delimiter=';',encoding = "ISO-8859-1",quoting=csv.QUOTE_ALL,error_bad_lines=False )

#include implicit recos only
books_ratings_implicit_df = books_ratings_df[books_ratings_df['Book-Rating'] == 0]
books_ratings_implicit_df = books_ratings_implicit_df.sort_values(by='User-ID')
books_ratings_implicit_df = books_ratings_implicit_df.sample(80000,random_state=113) #just sample for now
books_ratings_implicit_df = pd.merge(books_ratings_implicit_df,users_df[['User-ID']])
books_ratings_implicit_df = pd.merge(books_ratings_implicit_df,books_df[['ISBN']])
books_df = books_df[books_df['ISBN'].isin(books_ratings_implicit_df.ISBN.unique())]
users_df = users_df[users_df['User-ID'].isin(books_ratings_implicit_df['User-ID'].unique())]


#map the users and books to integer indicies
users_df = users_df[users_df['User-ID'].isin(books_ratings_implicit_df['User-ID'].unique())].drop_duplicates()
user_int_ids = np.array(range(0,len(users_df['User-ID'].unique())))
users_df['user_int_id']=user_int_ids

books_df = books_df[books_df['ISBN'].isin(books_ratings_implicit_df['ISBN'].unique())].drop_duplicates()
book_int_ids = np.array(range(0,len(books_df)))
books_df['book_int_id']=book_int_ids

books_ratings_implicit_df = pd.merge(books_ratings_implicit_df,books_df[['ISBN','book_int_id']])
books_ratings_implicit_df = pd.merge(books_ratings_implicit_df,users_df[['User-ID','user_int_id']])

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
/Users/yehiaelshater/anaconda/envs/mypython3/lib/python3.6/site-packages/I

In [155]:
books_ratings_implicit_df.head()

,User-ID,ISBN,Book-Rating,book_int_id,user_int_id
0,160819,0060509465,0,27029,9925
1,160819,0192823019,0,50366,9925
2,160819,055356451X,0,354,9925
3,160819,0425186059,0,19757,9925
4,160819,0679777504,0,17449,9925


In [156]:
interactions = Interactions(books_ratings_implicit_df['user_int_id'].values,books_ratings_implicit_df['book_int_id'].values)
train, test = random_train_test_split(interactions,test_percentage=0.2)
model = ImplicitFactorizationModel(n_iter=3,
                                   loss='bpr')
model.fit(train)
mrr = mrr_score(model, test)
mrr.mean()



0.006549693083536575

In [158]:
mrr = mrr_score(model, train)
mrr

array([0.61111111, 0.03225806, 1.        , ..., 0.17265274, 0.05555556,
       0.5       ])

In [144]:
p_r_score = precision_recall_score(model,train,k=5)


In [ ]:
p_r_score[0].mean()

In [128]:
print(len(books_ratings_implicit_df['user_int_id'].unique()) * 0.4)
print(len(mrr))
pd.DataFrame(mrr).describe()

1557.6000000000001
2026


,0
count,2026.000000
mean,0.002376
std,0.030120
min,0.000127
25%,0.000155
50%,0.000188
75%,0.000274
max,1.000000


In [ ]:
Interactions(np.array([0,1]),np.array([0,0]))

In [59]:
print(len(books_ratings_implicit_df['User-ID'].unique()))
print(len(users_df))
print(len(books_ratings_implicit_df['ISBN'].unique()))
len(books_ratings_implicit_df)

52451
52451
201623


647294

In [55]:

class WARP(Function): 
    '''
    autograd function of WARP loss
    '''
    @staticmethod
    def forward(ctx, input, target,positive_indices,negative_indices, max_num_trials = None):
    #def forward(ctx, input, target, max_num_trials = None):
        batch_size = target.size()[0]
        if max_num_trials is None:
            max_num_trials = target.size()[1] - 1
        
        #positive_indices = Variable(torch.zeros(input.size(),dtype=torch.float))
        #negative_indices = Variable(torch.zeros(input.size(),dtype=torch.float))

        L = torch.zeros(input.size()[0])
        
        all_labels_idx = np.arange(target.size()[1])
        
        Y = float(target.size()[1])
        J = torch.nonzero(target)
    
        for i in range(batch_size): 
            
            msk = np.ones(target.size()[1], dtype = bool)
            
            # Find the positive label for this example
            j = J[i, 1]
            positive_indices[i, j] = 1
            msk[j] = False
            
            # initialize the sample_score_margin
            sample_score_margin = -1
            num_trials = 0
            
            neg_labels_idx = all_labels_idx[msk]

            while ((sample_score_margin < 0) and (num_trials < max_num_trials)):
                 
                #randomly sample a negative label
                neg_idx = random.sample(list(neg_labels_idx), 1)[0]
                msk[neg_idx] = False
                neg_labels_idx = all_labels_idx[msk]
                
                num_trials += 1
                # calculate the score margin 
                sample_score_margin = 1 + input[i, neg_idx] - input[i, j] 
            
            if sample_score_margin < 0:
                # checks if no violating examples have been found 
                continue
            else: 
                loss_weight = np.log(math.floor((Y-1)/(num_trials)))
                L[i] = loss_weight
                negative_indices[i, neg_idx] = 1
        
        #print(positive_indices*input)
        #print(torch.sum(positive_indices*input, dim = 1))
        #print(negative_indices*input)

                
        loss = L * (1-torch.sum(positive_indices*input, dim = 1) + torch.sum(negative_indices*input, dim = 1))
        
        ctx.save_for_backward(input, target)
        ctx.L = L
        ctx.positive_indices = positive_indices
        ctx.negative_indices = negative_indices
        
        #print(positive_indices)
        #print(negative_indices)
        print(torch.sum(loss , dim = 0, keepdim = True))
        return torch.sum(loss , dim = 0, keepdim = True)

    # This function has only a single output, so it gets only one gradient 
    @staticmethod
    def backward(ctx, grad_output):
        input, target = ctx.saved_variables
        L = Variable(torch.unsqueeze(ctx.L, 1), requires_grad = False)
        #positive_indices = Variable(ctx.positive_indices, requires_grad = False) 
        #negative_indices = Variable(ctx.negative_indices, requires_grad = False)
        positive_indices = ctx.positive_indices
        negative_indices = ctx.negative_indices
        
        grad_input = grad_output*L*(negative_indices - positive_indices)

        return grad_input, None, None,None,None    

      
class WARPLoss(nn.Module): 
    def __init__(self,input_size, max_num_trials = None): 
        super(WARPLoss, self).__init__()
        self.max_num_trials = max_num_trials
        self.positive_indices = torch.nn.Parameter(torch.zeros(input_size),requires_grad=True)
        self.negative_indices = torch.nn.Parameter(torch.zeros(input_size),requires_grad=True)

        
    def forward(self, input, target): 
        return WARP.apply(input, target,self.positive_indices,self.negative_indices, self.max_num_trials)
        #return WARP.apply(input, target, self.max_num_trials)
    

In [63]:
#input_vector = Variable(torch.tensor([[0.35,0.63,0.59,0.76,0.17]],dtype=torch.float),requires_grad=True)
input_vector = Variable(torch.tensor([[0,0.63,0.59,0.76,1]],dtype=torch.float),requires_grad=True)
target_vector = Variable(torch.tensor([[1,0,1,0,1]],dtype=torch.float))
warp_module = WARPLoss(input_vector.size(),3)

optimizer = torch.optim.Adam(warp_module.parameters(), lr=0.000000000001)

for t in range(100):
    loss = warp_module(input_vector,target_vector)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

input_vector = Variable(torch.tensor([[0,0.63,0.59,0.76,1]],dtype=torch.float),requires_grad=True)    
predictions_output = warp_module(input_vector)
print(predictions_output)



tensor([ 2.2042])
tensor([ 3.2578])
tensor([ 3.2578])
tensor([ 3.2578])
tensor([ 4.1312])
tensor([ 4.1312])
tensor([ 5.5175])
tensor([ 5.5175])
tensor([ 5.5175])
tensor([ 5.5175])
tensor([ 5.5175])
tensor([ 5.5175])
tensor([ 5.5175])
tensor([ 5.5175])
tensor([ 5.5175])
tensor([ 5.5175])
tensor([ 5.5175])
tensor([ 5.5175])
tensor([ 5.5175])
tensor([ 5.5175])
tensor([ 5.5175])
tensor([ 5.5175])
tensor([ 5.5175])
tensor([ 5.5175])
tensor([ 5.5175])
tensor([ 5.5175])
tensor([ 5.5175])
tensor([ 5.5175])
tensor([ 5.5175])
tensor([ 5.5175])
tensor([ 5.5175])
tensor([ 5.5175])
tensor([ 5.5175])
tensor([ 5.5175])
tensor([ 5.5175])
tensor([ 5.5175])
tensor([ 5.5175])
tensor([ 5.5175])
tensor([ 5.5175])
tensor([ 5.5175])
tensor([ 5.5175])
tensor([ 5.5175])
tensor([ 5.5175])
tensor([ 5.5175])
tensor([ 5.5175])
tensor([ 5.5175])
tensor([ 5.5175])
tensor([ 5.5175])
tensor([ 5.5175])
tensor([ 5.5175])
tensor([ 5.5175])
tensor([ 5.5175])
tensor([ 5.5175])
tensor([ 5.5175])
tensor([ 5.5175])
tensor([ 5

/Users/yehiaelshater/anaconda/envs/mypython3/lib/python3.6/site-packages/ipykernel_launcher.py:77: DeprecationWarning: 'saved_variables' is deprecated; use 'saved_tensors'


TypeError: forward() missing 1 required positional argument: 'target'

In [84]:



batch_size = target_vector.size()[0]
max_num_trials = 10
if max_num_trials is None: 
    max_num_trials = target_vector.size()[1] - 1

positive_indices = torch.zeros(input_vector.size())
negative_indices = torch.zeros(input_vector.size())
L = torch.zeros(input_vector.size()[0])

all_labels_idx = np.arange(target_vector.size()[1])

i=0
Y = float(target_vector.size()[1])
J = torch.nonzero(target_vector)

msk = np.ones(target_vector.size()[1], dtype = bool)
            
# Find the positive label for this example
j = J[i, 1]
positive_indices[i, j] = 1
msk[j] = False



# initialize the sample_score_margin
sample_score_margin = -1
num_trials = 0

neg_labels_idx = all_labels_idx[msk]


neg_idx = random.sample(neg_labels_idx, 1)[0]
msk[neg_idx] = False
neg_labels_idx = all_labels_idx[msk]


TypeError: Population must be a sequence or set.  For dicts, use list(d).

In [55]:
J
positive_indices
neg_labels_idx

array([0, 1, 3, 4])

In [70]:
#random.sample(neg_labels_idx,0)
positive_indices.shape
random.sample(list(neg_labels_idx),1)

[4]